In [ ]:
import math
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib.ticker as mtick
from matplotlib.cm import get_cmap
# import addcopyfighandler

import seaborn as sns

from IPython.core.display import HTML

sys.path.append("../RD-systems-and-test-benches/utils")
import data_processing as dp

import model_fsolve as modf

import scipy.optimize as sco
import scipy.interpolate as sci

import simu as sim

import hx_hydraulic as hxhy
import from_excel as fe
import data_plots as dplt

import fluids as fds
from CoolProp.CoolProp import PropsSI

import openpyxl as opxl

import torch
import torch_model as modto

In [ ]:
path = r"G:\Drive partagés\BU04-Innovation\PVT-PL-model\Inputs"
file_name = r"\García_Guendulain.xlsx"

hx, par, condi0 = fe.initialize(path, file_name)

In [ ]:
# par['D_riser'] = 0.0054
# par['D_man'] = 4*par['D_riser']

In [ ]:
# Test d'un seul débit
tabl, res, PL,u,K = modf.PL_fsolve(par,condi0,print=False)


In [ ]:
list_rd = [0.25, 0.5, 1.]
dplt.u_rd(par,condi0,list_rd,'uin')
dplt.u_rd(par,condi0,list_rd,'ux')
dplt.u_rd(par,condi0,list_rd,'uout')
dplt.K_rd(par,condi0,list_rd,'Ky_in')
dplt.K_rd(par,condi0,list_rd,'Ky_out')
dplt.K_rd(par,condi0,list_rd,'Kx_in')
dplt.K_rd(par,condi0,list_rd,'Kx_out')

In [ ]:
K

In [ ]:
par['ref']

In [ ]:
dplt.PL_percent(PL)

In [ ]:
dplt.PL_hist(PL)

In [ ]:
dplt.beta_riser(tabl, axis=[0, 8, 0.108, 0.116])

In [ ]:
list_Vdot = np.array([i*60 for i in range(1,10)])

In [ ]:
list_rd = [0.25, 0.5, 1.]
dplt.PL_ratio(par,condi0,list_Vdot,list_rd)

In [ ]:
for rd in list_rd:
    modf.change_diameter(par, par['D_riser']/rd, name='man')
    par['D_man']=par['D_riser']/rd
    par['A_man']=par['A_riser']/rd**2
    tabl, res, PL, u, K = modf.PL_fsolve(par,condi0,print=False)
    dplt.PL_hist(PL)
    print(PL)

In [ ]:
return coeff*fds.fittings.K_branch_diverging_Crane(D_run=D_run, D_branch=D_branch, Q_run=Q_run, Q_branch=Q_branch, angle=theta)

In [ ]:
PL_list_MICOE = np.array([485, 977, 1336, 1941, 2800])
plt.scatter(list_Vdot, list_PL/1000, label='MICOE model')
plt.scatter(list_Vdot, PL_list_MICOE/1000, label='MICOE datasheet')
plt.axis([0,1200, 0, 3])
plt.xlabel('Flow rate [L/h]')
plt.ylabel('Pressure loss [kPa]')
plt.legend()